In [24]:
import pandas as pd
import numpy as np


In [18]:
genreAll = pd.read_csv("/home/ibroto/Documents/UPF_SMC/ML/Lab1/Lab1/GenreAll.csv")
genreAll.rename(columns={' genre':'genre'}, inplace = True)
feat_30 = pd.read_csv("/home/ibroto/Documents/UPF_SMC/ML/Lab1/Lab1/features_30_sec.csv")
feat_3 = pd.read_csv("/home/ibroto/Documents/UPF_SMC/ML/Lab1/Lab1/features_3_sec.csv")


Data Overview

In [ ]:
nrows, ncols = genreAll.shape
print(f"Nbr of observations: {nrows-1}\nNbr of features: {ncols-1}\nNbr of classes: {pd.unique(genreAll.genre).size}")




Nbr of observations: 992
Nbr of features: 240
Nbr of classes: 10


In [22]:
pd.unique(genreAll.genre).size


10

In [14]:
genreAll.columns

Index(['highlevel_danceability_value', ' highlevel_equalization_profile_value',
       ' highlevel_excitement_value', ' highlevel_intensity_value',
       ' highlevel_speech_music_value', ' highlevel_voice_instrumental_value',
       ' loudness_dynamic_complexity_dvar',
       ' loudness_dynamic_complexity_mean', ' loudness_dynamic_complexity_var',
       ' loudness_larm_dvar',
       ...
       ' tonal_chords_strength_dvar', ' tonal_chords_strength_mean',
       ' tonal_chords_strength_var', ' tonal_dissonance_dvar',
       ' tonal_dissonance_mean', ' tonal_dissonance_var',
       ' tonal_key_mode_value', ' tonal_key_strength_value',
       ' tonal_tuning_equal_tempered_deviation_value', ' genre'],
      dtype='object', length=241)

Train-test split

In [27]:
genreAll[genreAll.columns[:-1]]

,highlevel_danceability_value,highlevel_equalization_profile_value,highlevel_excitement_value,highlevel_intensity_value,highlevel_speech_music_value,highlevel_voice_instrumental_value,loudness_dynamic_complexity_dvar,loudness_dynamic_complexity_mean,loudness_dynamic_complexity_var,loudness_larm_dvar,...,tonal_chords_number_rate_value,tonal_chords_strength_dvar,tonal_chords_strength_mean,tonal_chords_strength_var,tonal_dissonance_dvar,tonal_dissonance_mean,tonal_dissonance_var,tonal_key_mode_value,tonal_key_strength_value,tonal_tuning_equal_tempered_deviation_value
0,0.477744,0.372766,0.029087,0.093526,speech,instrumental,-0.366526,0.832351,-0.398487,0.262413,...,-0.250890,0.175405,0.101673,0.989475,0.167484,-0.091051,0.085449,major,0.162259,2.423830
1,0.271161,0.372766,-1.283803,-0.615420,speech,voice,0.275184,1.901008,-0.287263,0.078360,...,0.134622,-0.157583,-0.652126,-0.992276,0.606661,-0.468630,0.416914,major,0.230571,-0.207952
2,0.089011,0.372766,0.029087,1.511419,music,instrumental,-0.177390,0.046932,0.035041,-0.157974,...,0.134622,-0.001059,-0.220255,0.978014,-0.236338,0.470580,-0.205887,major,1.024735,-0.211034
3,0.243503,0.372766,0.029087,-0.615420,music,instrumental,-0.319420,-0.953075,0.239607,-0.659566,...,-0.636402,0.214463,0.853728,1.327351,-0.542341,0.642223,-0.791110,major,0.079033,0.148357
4,0.594883,-1.940715,-1.283803,0.093526,speech,instrumental,-0.224956,1.154507,-0.420087,0.281666,...,-1.021915,-0.140930,-0.080519,-1.163700,0.094741,-0.164770,-0.277919,major,-0.337966,-0.277459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,-1.427786,0.372766,-1.283803,0.802473,music,instrumental,0.559649,-0.183165,0.622257,0.084044,...,-0.636402,-0.366172,0.318132,0.748104,-0.745670,-0.226276,-0.095535,major,1.322209,0.061931
989,-0.078540,0.372766,1.341977,0.802473,music,voice,-0.108340,-0.584670,0.119356,-0.314633,...,-1.021915,-0.284281,0.096984,-0.089204,-1.394305,1.206240,-1.225032,major,0.498037,-0.269263
990,-1.045058,0.372766,-1.283803,0.093526,music,instrumental,-0.156503,-0.975097,0.297968,-0.315637,...,-0.250890,-0.408811,-1.467413,-0.965934,-0.713245,0.178487,0.002003,major,-0.624586,-1.079980
991,-0.227043,0.372766,-1.283803,0.802473,music,instrumental,-0.125633,-1.068999,0.274430,-0.725643,...,-0.636402,-0.329513,1.205735,-0.789754,-0.415943,0.386272,-0.764152,major,0.417567,-0.522861


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(genreAll[genreAll.columns[:-1]], genreAll.genre, test_size=0.33, random_state=42)
X_train


,highlevel_danceability_value,highlevel_equalization_profile_value,highlevel_excitement_value,highlevel_intensity_value,highlevel_speech_music_value,highlevel_voice_instrumental_value,loudness_dynamic_complexity_dvar,loudness_dynamic_complexity_mean,loudness_dynamic_complexity_var,loudness_larm_dvar,...,tonal_chords_number_rate_value,tonal_chords_strength_dvar,tonal_chords_strength_mean,tonal_chords_strength_var,tonal_dissonance_dvar,tonal_dissonance_mean,tonal_dissonance_var,tonal_key_mode_value,tonal_key_strength_value,tonal_tuning_equal_tempered_deviation_value
316,0.467545,-0.783974,1.341977,0.802473,speech,voice,0.671042,0.867836,0.142840,-0.599100,...,-1.023697,-0.043184,0.747260,0.154253,-0.213795,0.549014,0.294885,major,-0.161714,0.107154
887,0.226115,0.372766,1.341977,-0.615420,music,voice,-0.594318,-0.786017,-1.174160,-0.388638,...,-1.792343,0.112178,1.017516,-0.074750,-0.032360,0.153537,-0.114415,major,1.442675,-0.523726
544,-1.286749,0.372766,-1.283803,-2.033312,music,instrumental,2.184066,0.053222,0.498727,0.323659,...,2.447694,-0.118493,-0.737289,0.976675,-0.252878,-1.362091,2.715179,minor,-1.027723,0.497932
465,1.552020,-1.940715,0.029087,0.093526,music,voice,-0.412341,-0.234239,-0.103300,-0.377244,...,0.520134,1.375822,-1.184224,-0.750849,0.546644,0.117770,0.313968,minor,-2.254605,0.999331
529,-0.190936,0.372766,0.029087,0.802473,music,instrumental,-0.044563,-0.383867,-0.005705,-0.373108,...,0.520134,-0.057034,-0.615155,0.584375,-0.486300,0.004930,-0.422724,major,-0.518622,-0.727738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-1.099677,0.372766,-1.283803,-0.615420,speech,instrumental,2.131936,1.646857,2.496069,1.989191,...,1.291158,-0.252065,-0.320488,0.189112,0.416630,-1.619716,0.722733,minor,-0.326020,-0.947985
270,-0.341693,0.372766,0.029087,-0.615420,music,voice,-0.103055,-0.035755,-0.141779,-0.402013,...,-0.636402,-0.104509,0.994302,0.076086,-0.146236,0.140413,-0.370140,major,0.626151,-0.213018
860,0.057781,0.372766,1.341977,-1.324366,speech,instrumental,0.717954,0.809965,0.205012,2.263062,...,-0.634019,-0.006853,2.121189,0.907540,1.437929,-0.548092,0.241808,major,1.224564,0.825132
435,2.769760,0.372766,0.029087,0.093526,music,voice,-0.407429,-0.390595,-0.020312,-0.411278,...,0.905646,0.357111,-1.917093,-0.062836,-0.389075,0.718669,-0.762411,major,-3.161648,0.531784


In [10]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)